In [6]:
"""
Scroll and Scrape main application file.
"""
import time
import urllib.parse

from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
browser = None

# browser = webdriver.Chrome()

MAX_STOPS = 3


def scroll(url,_map):
    """
    Scroll through HTML page for given URL.

    Stop scrolling if height has not increased since last scroll.
    This might also mean that the connection has stopped and the script is
    not aware of the connection error.

    When the end appears to be reached, add in a check to wait a bit and then
    check again a few times, to check we are really at the end. Reset the count
    whenever we have scrolled successfully.
    """
    browser.get(url)
    time.sleep(2)

    current_height = browser.execute_script("return document.body.scrollHeight")

    stops = 0
    
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        new_height = browser.execute_script("return document.body.scrollHeight")
        
        if new_height == current_height:
            stops += 1
            handleTweet(_map)
            if stops == MAX_STOPS:
                break
            time.sleep(5)
        else:
            handleTweet(_map)
            current_height = new_height
            stops = 0
def handleTweet(_map):
    tweets = browser.find_elements_by_tag_name("article")
    print('# tweet', len(tweets))
    for tweet in tweets:
        tweet_id = tweet.find_element_by_xpath("//time").find_element_by_xpath("./..").get_attribute("href")
        time = tweet.find_element_by_xpath("//time").text
        _map[str(tweet_id)] = time
    return _map

def getUrl(source,_from,_to):    
    text_query = f'From:{source} since:{_from} until:{_to}'
    # Convert characters to be websafe. In particular spaces and punctuation, but keep brackets.
    encoded_query = urllib.parse.quote(text_query, safe="()")
    
    base_url = "https://twitter.com/search?src=typed_query&f=tweets&q="
    # base_url = "https://twitter.com/search?q="
    url = f"{base_url}{encoded_query}"

def start(source,_from,_to):
    if(browser != None):
        browser.quit()
    browser = webdriver.Chrome(options=options)
    _map = {}
    scroll(getUrl(source,_from,_to),_map)
    browser.quit()
    return _map

In [7]:
import pandas as pd
from getTopic import handleData
from getTopic import df2tweet
from tqdm.auto import tqdm
from datetime import datetime
import json

listNews = pd.read_excel("data/News List and dictionary.xlsx",sheet_name='News list')
listNews = listNews[pd.isna(listNews["Type"])]

def getFromsource(_from,_to):
    for i in tqdm(range(0,listNews.shape[0])):
        try:
            news = listNews.iloc[i]
            user_name = news['Twitter Link'].split('/')[-1]
            start(user_name,_from,_to)
        except:
            print('fail')

In [8]:
fileName = 'FNsample1'
df = pd.read_csv("../../../data/raw/excel_files/"+fileName+".csv")
df

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,720076604769079296,2753211010,Phoenix Daily News,PhoenixDailyNew,"City of Phoenix, Arizona","Local news, sports, business, politics, entert...",NaN,18574,7903,2014-08-28,...,absent,absent,0.0,0.0,1.0,2.0,[],[http://www.azcentral.com/story/news/local/pho...,NaN,NaN
1,720463270512062464,2630842499,San Diego Daily,DailySanDiego,"San Diego, CA","Follow for San Diego's breaking news, special ...",NaN,22715,8255,2014-06-22,...,absent,absent,0.0,0.0,0.0,0.0,[politics],[],NaN,NaN
2,721415092831543296,2587843805,KansasCityDailyNews,KansasDailyNews,"Kansas City, MO","Local news, sports, business, politics, entert...",NaN,29357,6720,2014-06-06,...,absent,absent,0.0,0.0,1.0,1.0,[news],[http://bit.ly/1r3CV1I],NaN,NaN
3,721467000000000000,2928870434,Newspeak Daily,NewspeakDaily,NaN,it's never too late to reinvent the bicycle,NaN,5816,865,12/13/2014,...,absent,absent,0.0,0.0,0.0,0.0,[politics],[],NaN,NaN
4,719489604454211584,2601235821,Pittsburgh Today,TodayPittsburgh,"Pittsburgh, PA",Pittsburgh's local news on Twitter. Breaking n...,NaN,20790,10408,2014-06-11,...,absent,absent,0.0,0.0,0.0,0.0,[news],[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,721211355009691648,2495567768,San Francisco Daily,DailySanFran,"San Francisco, CA","Follow for San Francisco's breaking news, spec...",NaN,22688,8375,2014-05-15,...,absent,absent,0.0,0.0,1.0,1.0,[],[http://bit.ly/1p63b9z],NaN,NaN
996,720289237250117632,2494112058,San Jose Daily,DailySanJose,USA,"Follow for San Jose's breaking news, special r...",NaN,20135,9313,2014-05-14,...,absent,absent,0.0,0.0,0.0,0.0,[politics],[],NaN,NaN
997,719978207898787840,2587843805,KansasCityDailyNews,KansasDailyNews,"Kansas City, MO","Local news, sports, business, politics, entert...",NaN,29357,6720,2014-06-06,...,absent,absent,0.0,0.0,0.0,0.0,[news],[http://bit.ly/1UZjOC3],NaN,NaN
998,719905252732903424,2495567768,San Francisco Daily,DailySanFran,"San Francisco, CA","Follow for San Francisco's breaking news, spec...",NaN,22688,8375,2014-05-15,...,absent,absent,0.0,0.0,0.0,1.0,[news],[],NaN,NaN
